In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import logging
import time
import datetime
from dateutil.relativedelta import relativedelta
import shutil
import imghdr
import random
from flask import Flask
import urllib.request
import smtplib, ssl
from email.message import EmailMessage
from flask import request, Response
import glob
from selenium.webdriver.common.keys import Keys
logging.basicConfig(handlers=[logging.FileHandler('filmesErrados.log', 'w', 'utf-8')])
from datetime import date
from dateutil.relativedelta import relativedelta
import database as db

In [61]:
from importlib import reload # reload 
reload(db)
db.drop_table('SixMonthsBefore')
db.drop_table('SixMonthsAfter')

Conectado com sucesso!
Conexão encerrada com sucesso!
Conectado com sucesso!
Conexão encerrada com sucesso!


In [62]:
db.create_table('SixMonthsBefore')
db.create_table('SixMonthsAfter')


Conectado com sucesso!
Tabela SixMonthsAfter criada!
Conexão encerrada com sucesso!
Conectado com sucesso!
Tabela SixMonthsAfter criada!
Conexão encerrada com sucesso!


In [3]:
options = Options()
options.add_argument('--headless') 
prefs = {'download.default_directory': os.getcwd()}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\anton\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


##### Realizando scrap em todos os filmes do site https://www.adorocinema.com/filmes-todos/

In [3]:
driver.get("https://www.adorocinema.com/filmes-todos/")
paginas = driver.find_elements(By.XPATH, "//a[@class='xXx button button-md item']")
coleta = []
for x in range(1, int(paginas[-1].text)+1):
    print(f'Página: {x}')
    driver.get(f"https://www.adorocinema.com/filmes-todos/?page={x}")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    info = soup.findAll('div', class_='meta-body-item meta-body-info')
    title = soup.findAll('a', class_='meta-title-link')
    for filme in range(len(info)):
        info[filme] = info[filme].text.split('/')
        if len(info[filme]) != 3:
            log = [title[filme].text, info[filme]]
            logging.warning(f'filme {log}')
        try:
            info[filme] = list(map(lambda x: x.replace("\n", ""), info[filme]))
            info[filme].insert(0, title[filme].text)
            coleta.append([info[filme][0], info[filme][1], info[filme][-1]])
        except:
            logging.critical(f'{info[filme]}')

Página: 1
Página: 2
Página: 3
Página: 4
Página: 5
Página: 6
Página: 7
Página: 8
Página: 9
Página: 10
Página: 11
Página: 12
Página: 13
Página: 14
Página: 15
Página: 16
Página: 17
Página: 18
Página: 19
Página: 20
Página: 21
Página: 22
Página: 23
Página: 24
Página: 25
Página: 26
Página: 27
Página: 28
Página: 29
Página: 30
Página: 31
Página: 32
Página: 33
Página: 34
Página: 35
Página: 36
Página: 37
Página: 38
Página: 39
Página: 40
Página: 41
Página: 42
Página: 43
Página: 44
Página: 45
Página: 46
Página: 47
Página: 48
Página: 49
Página: 50
Página: 51
Página: 52
Página: 53
Página: 54
Página: 55
Página: 56
Página: 57
Página: 58
Página: 59
Página: 60
Página: 61
Página: 62
Página: 63
Página: 64
Página: 65
Página: 66
Página: 67
Página: 68
Página: 69
Página: 70
Página: 71
Página: 72
Página: 73
Página: 74
Página: 75
Página: 76
Página: 77
Página: 78
Página: 79
Página: 80
Página: 81
Página: 82
Página: 83
Página: 84
Página: 85
Página: 86
Página: 87
Página: 88
Página: 89
Página: 90
Página: 91
Página: 

In [4]:
meses = {"de janeiro de":'01',
         "de fevereiro de":'02',
         "de março de":'03',
         "de abril de":'04',
         "de maio de":'05',
         "de junho de":'06',
         "de julho de":'07',
         "de agosto de":'08',
         "de setembro de":'09',
         "de outubro de":'10',
         "de novembro de":'11',
         "de dezembro de":'12',
         "janeiro":'01',
         "fevereiro":'02',
         "março":'03',
         "abril":'04',
         "maio":'05',
         "junho":'06',
         "julho":'07',
         "agosto":'08',
         "setembro":'09',
         "outubro":'10',
         "novembro":'11',
         "dezembro":'12'}


##### criando um df com todos os filmes

In [145]:
df = pd.DataFrame(coleta, columns=['title', 'release_date', 'genre'])

##### removendo filmes em que as datas não estão no formato correto

In [146]:
filmesEmBreve = df.loc[df['release_date'] == 'em breve']
filmesDesconhecidos = df.loc[df['release_date'] == 'Data de lançamento desconhecida']

##### dropando nulos

In [147]:
df.drop(index=filmesEmBreve.index, inplace=True)
df.drop(index=filmesDesconhecidos.index, inplace=True)

##### Corrigindo o formato da coluna "data"

In [ ]:
def dataTransformer(lista):
    """Função que transforma o dado do tipo string para datetime"""
    for data in range(len(lista)):
        if 'breve' in lista[data]:
            lista[data] = np.nan
        else:
            if 'No cinema' in lista[data]:
                lista[data].replace('No cinema', '')
            for key, value in meses.items():
                lista[data] = lista[data].replace(key, value)
            if len(lista[data].split(' ')) == 3:
                lista[data] = lista[data].replace(' ', '/')
                lista[data] = lista[data].split('/')
                for i in range(len(lista[data])):
                    if len(lista[data][i]) == 1:
                        lista[data][i] = '0'+lista[data][i]
                lista[data] = '/'.join(lista[data])
            else:
                lista[data] = np.nan
    return lista

data = (dataTransformer(df['release_date'].values.tolist()))
df['release_date'] = data

##### dropando nulos

In [149]:
df.dropna(inplace=True)

##### transformando 'data' em datetime

In [150]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%d/%m/%Y')

##### função que limita o dataset em 1 ano, 6 meses antes até 6 meses dapois do dia de hoje

In [4]:
def rangeDate(frame, months):
    """Função que limita o dataframe em um ano, 6 meses antes de depois do dia de hoje"""
    today = datetime.date.today()
    sixMonthsBefore = today + relativedelta(months=-months)
    sixMonthsAfter = today + relativedelta(months=months)
    return frame.loc[frame['release_date'].between(str(sixMonthsBefore), str(sixMonthsAfter))]

In [17]:
filmes = pd.read_csv('filmes.csv')
filmes_ordenados = filmes.sort_values(by=['release_date'],  ascending=True)
filmes_ordenados.head(10)

,title,release_date,genre
750,Carreiras,2021-11-14,Outros
710,Amigas Improváveis,2021-11-14,"Drama,Comédia"
771,Coração Violento,2021-11-15,"Romance,Drama,Suspense"
246,O Quarto Secreto,2021-11-15,"Suspense,Ficção científica"
845,Kuroko's Basketball: Last Game,2021-11-15,"Animação,Drama,Esporte"
774,Dahl e Potter: um Encontro Mágico,2021-11-15,"Comédia dramática,Família,Drama"
821,Front And Center Presents: Jack Johnson,2021-11-16,Show
273,Chamados do Além,2021-11-16,Terror
622,Policiando A Polícia 2020,2021-11-16,Documentário
313,À Prova de Balas,2021-11-16,"Ação,Comédia"


In [63]:
for index, row in filmes_ordenados.iterrows():
    if row['release_date'] >= str(date.today()) and row['release_date']  <= str(date.today() + relativedelta(months=+6)):
        nome_table = 'SixMonthsAfter'
        db.inserir_dados(nome_table, row['title'], row['genre'], row['release_date'])
    elif row['release_date'] < str(date.today()) and row['release_date']  >= str(date.today() + relativedelta(months=-6)):
        nome_table = 'SixMonthsBefore'
        db.inserir_dados(nome_table, row['title'], row['genre'], row['release_date'])
    else:
        nome_table = None

Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidade linhas adicionadas:  1
Conexão encerrada com sucesso!
Conectado com sucesso!
Quantidad

##### Função que antecipa o filme que estreiará daqui a um mes

In [5]:
def moviePremiereNotification(frame, months):
    """Função que verifica os filmes que lançarão daqui a um mês"""
    today = datetime.date.today()
    monthsLater = today + relativedelta(months=months)
    return frame.loc[frame['release_date'] == str(monthsLater)]['title'].values.tolist()

##### Procurando imagem do filme no google

In [6]:
def searchImage(key, numImages=1):
    """Função que procura imagens relacionadas ao filme no google images"""
    driver.get("https://images.google.com/")
    search = driver.find_element_by_name('q')
    search.send_keys(str(key) + ' filme')
    time.sleep(0.2)
    search.send_keys(Keys.ENTER)
    images = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')

    links = []
    for link in images:
        links.append(link.get_property('src'))

    path = 'images/'
    exist = os.path.exists(path)
    if not exist:
        os.makedirs(path)

    for images in range(numImages):
        try:
            urllib.request.urlretrieve(str(links[images]), f"images/{key}{images}.jpg")
        except:
            logging.warning(f"Não foi possível baixar a imagem do filme {key}")
    return path

##### função que manda o email para o empresário

In [7]:
def sendEmail(movie, email, imagePath):
    """Função que envia o email com informações do filme como nome e imagens"""
    senderEmail = "emailsenderfiap@gmail.com"
    receiverEmail = email
    password = "emailsender325523"
    
    newMessage = EmailMessage()
    newMessage['Subject'] = "Novo filme na área!!"
    newMessage['From'] = senderEmail                   
    newMessage['Para'] = receiverEmail
    newMessage.set_content(f"Falta apenas um mês para o filme {movie} ser lançado oficialmente, não esqueça de fazer a divulgação! Aqui estão algumas imagens para você utilizar como banner!")
    try:
        images = glob.glob(imagePath + "/*")
        for image in images:
            with open(image, 'rb') as f:
                imageData = f.read()
                imageType = imghdr.what(f.name)
                nomeImagem = f.name
            newMessage.add_attachment(imageData, maintype='image', subtype=imageType, filename=nomeImagem)
    except:
        logging.critical('houve um erro ao anexar as imagens ao email')

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
        smtp.login(senderEmail, password)
        smtp.sendmail(senderEmail, receiverEmail, newMessage.as_string())

##### juntando as funções de procurar imagem, antecipar estréia e mandar email para o empresário

In [8]:
def notification(movieList, nImagens, email):
    """Função que junta as funcionalidades de procurar imagem,
    encontrar filmes que lançarão daqui a um mês e mandar email para o empresário"""
    if len(movieList) > 1:
        for movie in movieList:
            path = searchImage(f'{movie[0]}', nImagens)
            sendEmail(f'{movie[0]}', email, imagePath=path)
            shutil.rmtree(path=path)
    else:
        path = searchImage(f'{movieList[0]}', nImagens)
        sendEmail(f'{movieList[0]}', email, imagePath=path)
        shutil.rmtree(path=path)

##### pocura filmes do mesmo gênero do filme passado

In [20]:
def sameGenreMovies(df, movie, nMovies=5):
    """Retorna um dicionário com filmes do mesmo genero"""
    genreArray = []
    genres = movie['genre'][0].replace(' ', '').split(',')
    for genre in genres:
        genreArray.append(df[df['genre'].str.contains(f'{genre}')]['title'].values)
    
    # Chose a random movie from the same genre
    suggestion = []
    for movieGenre in genreArray:
        for i in range(nMovies):
            suggestion.append(random.choice(movieGenre))
    dictFormat = {"suggestion": suggestion}
    return dictFormat

##### Criando API

A api recebe um JSON com informações sobre o filme que o usuário quer assistir:

{

        "title":"Doutor Estranho",

        "genre":"Fantasia,Ação, Aventura"
    
}

Se tudo ocorrer bem, é retornado um JSON para o usuário com filmes que se encaixam na mesma categoria

In [64]:
app = Flask(__name__)

@app.route("/", methods=['PUT'])
def buyTicket():
    """Função que incrementa uma nova compra de ingresso no banco de dados"""
    movie = request.json
    before = db.read_table('SixMonthsBefore')
    df_before = pd.DataFrame(before, columns=['indexador', 'title','genre', 'release_date', 'n_orders'])
    if len(df_before[df_before['title'].str.contains(movie['title'])]):
        # Incrementa o valor no banco
        db.update_orders(movie['title'])
        sameGen = sameGenreMovies(df_before[['title','genre', 'release_date']], movie, nMovies=5)
        return sameGen
    else:
        logging.warning(f'O usuário informou um filme que não está em cartaz')
        return "Desculpa, não encontramos esse filme"

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Conexão encerrada com sucesso!
Conexão encerrada com sucesso!


# Iniciando os teste

In [6]:
df = pd.read_csv('filmes.csv')
df = rangeDate(df, 6)
df

,title,release_date,genre
0,Doutor Estranho no Multiverso da Loucura,2022-05-05,"Fantasia,Ação, Aventura"
1,O Homem do Norte,2022-05-12,"Ação,Histórico"
2,365 Dias: Hoje,2022-04-27,"Drama,Romance,Erótico"
3,Sonic 2 - O Filme,2022-04-07,"Animação,Família,Aventura"
4,Jujutsu Kaisen 0: O Filme,2022-04-28,"Animação,Ação,Drama"
...,...,...,...
1049,Wild - Rede selvagem,2021-12-02,Documentário
1050,Women Of Troy,2022-01-14,Documentário
1051,Yalda - Uma Noite de Perdão,2021-12-09,"Drama,Suspense"
1052,Yesterday Wonder I Was,2021-11-29,"Drama,Ficção científica"


dataTransformer(lista)

rangeDate(frame, months)

moviePremiereNotification(frame, months)

searchImage(key, numImages=1)

sendEmail(movie, email, imagePath)

notification(movieList, nImagens, email)

sameGenreMovies(df, movie, nMovies=5)